# Day 9: Rope Bridge

## Part 1

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [1]:
# Loading in rope motions from input file
with open('input.txt', 'r') as f:
    rope_motions = f.read().splitlines()

In [2]:
def check_vicinity(x_head, y_head, x_tail, y_tail):
    '''
    Checks if the head is in the vicinity of the tail
    
    Inputs:
        - x_head (int): The x-coordinate of the head
        - y_head (int): The y-coordinate of the head
        - x_tail (int): The x-coordinate of the tail
        - y_tail (int): The y-coordinate of the tail
        
    Returns:
        - is_in_vicinity (bool): True if is in the vicinity; False if not in the vicinity
    '''
    # Instantiating a value to represent the vicinity boolean
    is_in_vicinity = False
    
    # Iterating over a 3x3 grid centered over the tail position
    for x in range(x_tail - 1, x_tail + 2):
        
        # Breaking loop if boolean set in nested for loop
        if is_in_vicinity:
            break
            
        for y in range(y_tail -1, y_tail + 2):
            
            # Checking if the x- and y-coordinate positions match the head
            if x == x_head and y == y_head:
                
                # Setting the boolean to be true in the case where a match is found
                is_in_vicinity = True
                
                # Breaking the loop
                break
            else:
                # Setting the boolean to be true in the case where a match is not found
                is_in_vicinity = False
                
    return is_in_vicinity

In [3]:
def move_tail(x_head, y_head, x_tail, y_tail, direction):
    '''
    Moves the position of the tail appropriately
    
    Inputs:
        - x_head (int): The x-coordinate position of the head
        - y_head (int): The y-coordinate position of the head
        - x_tail (int): The x-coordinate position of the tail
        - y_tail (int): The y-coordinate position of the tail
        - direction (str): Which direction the tail needs to head
    Returns:
        - new_x_tail (int): The new x-coordinate position of the tail
        - new_y_tail (int): The new y-coordinate position of the tail
    '''
    
    # Handing scenario where the direction is up
    if direction == 'U':
        
        # Getting the new y-coordinate
        new_y_tail = y_tail + 1
        
        # Getting the appropriate new x-coordinate
        if (x_tail - 1) == x_head:
            new_x_tail = x_tail - 1
        elif x_tail == x_head:
            new_x_tail = x_tail
        elif (x_tail + 1) == x_head:
            new_x_tail = x_tail + 1
            
    # Handing scenario where the direction is down
    if direction == 'D':
        
        # Getting the new y-coordinate
        new_y_tail = y_tail - 1
        
        # Getting the appropriate new x-coordinate
        if (x_tail - 1) == x_head:
            new_x_tail = x_tail - 1
        elif x_tail == x_head:
            new_x_tail = x_tail
        elif (x_tail + 1) == x_head:
            new_x_tail = x_tail + 1
    
    # Handing scenario where the direction is left
    if direction == 'L':
        
        # Getting the new y-coordinate
        new_x_tail = x_tail - 1
        
        # Getting the appropriate new x-coordinate
        if (y_tail - 1) == y_head:
            new_y_tail = y_tail - 1
        elif y_tail == y_head:
            new_y_tail = y_tail
        elif (y_tail + 1) == y_head:
            new_y_tail = y_tail + 1
            
    # Handing scenario where the direction is right
    if direction == 'R':
        
        # Getting the new y-coordinate
        new_x_tail = x_tail + 1
        
        # Getting the appropriate new x-coordinate
        if (y_tail - 1) == y_head:
            new_y_tail = y_tail - 1
        elif y_tail == y_head:
            new_y_tail = y_tail
        elif (y_tail + 1) == y_head:
            new_y_tail = y_tail + 1
    
    return new_x_tail, new_y_tail

In [4]:
def move_head(x_head, y_head, direction):
    '''
    Moves the position of the head appropriately
    
    Inputs:
        - x_head (int): The x-coordinate position of the head
        - y_head (int): The y-coordinate position of the head
        - direction (str): Which direction the tail needs to head
    Returns:
        - new_x_head (int): The new x-coordinate position of the head
        - new_y_head (int): The new y-coordinate position of the head
    '''
    
    # Handing scenario where the direction is up
    if direction == 'U':
        
        # Setting the coordinate values approriately
        new_x_head = x_head
        new_y_head = y_head + 1
        
    # Handing scenario where the direction is down
    if direction == 'D':
        
        # Setting the coordinate values approriately
        new_x_head = x_head
        new_y_head = y_head - 1
        
    # Handing scenario where the direction is left
    if direction == 'L':
        
        # Setting the coordinate values approriately
        new_x_head = x_head - 1
        new_y_head = y_head
        
    # Handing scenario where the direction is right
    if direction == 'R':
        
        # Setting the coordinate values approriately
        new_x_head = x_head + 1
        new_y_head = y_head
        
    return new_x_head, new_y_head

In [5]:
# Instantiating values to represent the x- and y-coordinates of the head and tail
x_head, y_head, x_tail, y_tail = 0, 0, 0, 0

# Creating a list to note all the positions visited
positions_visited = []

# Iterating over all the rope motions
for motion in rope_motions:
    
    # Splitting the direction of the motion from number of units to move
    direction, num_units = motion.split(' ')
    
    # Going through the motions (hee hee)
    for _ in range(int(num_units)):
        
        # Moving the head to its new position
        x_head, y_head = move_head(x_head, y_head, direction)
        
        # Checking if tail is still in the vicinity of the head
        if not check_vicinity(x_head, y_head, x_tail, y_tail):
            
            # Moving the tail appropriately if not in the vicinity of the head
            x_tail, y_tail = move_tail(x_head, y_head, x_tail, y_tail, direction)
            
            # Checking to see if this position has been visited
            if (x_tail, y_tail) not in positions_visited:
                
                # Appending unvisited positions
                positions_visited.append((x_tail, y_tail))
                
# Printing the final result
print(f'Total number of places visited by the tail: {len(positions_visited)}')

Total number of places visited by the tail: 5960


## Part 2

A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

```

== Initial State ==

......
......
......
......
H.....  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)

== R 4 ==

......
......
......
......
1H....  (1 covers 2, 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
21H...  (2 covers 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
321H..  (3 covers 4, 5, 6, 7, 8, 9, s)

......
......
......
......
4321H.  (4 covers 5, 6, 7, 8, 9, s)

== U 4 ==

......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)

......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)

......
....H.
....1.
.432..
5.....  (5 covers 6, 7, 8, 9, s)

....H.
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

== L 3 ==

...H..
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

..H1..
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

.H1...
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

..1...
.H.2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== R 4 ==

..1...
..H2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

..1...
...H..  (H covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...1H.  (1 covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21H
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

......
...21.
..43.H
.5....
6.....  (6 covers 7, 8, 9, s)

== L 5 ==

......
...21.
..43H.
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21.
..4H..  (H covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
..H1..  (H covers 4; 1 covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
.H13..  (1 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
H123..  (2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

== R 2 ==

......
......
.H23..  (H covers 1; 2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
.1H3..  (H covers 2, 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```

Now, you need to keep track of the positions the new tail, 9, visits. In this example, the tail never moves, and so it only visits 1 position. However, be careful: more types of motion are possible than before, so you might want to visually compare your simulated rope to the one above.

Here's a larger example:

```
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20

These motions occur as follows (individual steps are not shown):
```

```
== Initial State ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........H..............  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== R 5 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........54321H.........  (5 covers 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== U 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
................H.........
................1.........
................2.........
................3.........
...............54.........
..............6...........
.............7............
............8.............
...........9..............  (9 covers s)
..........................
..........................
..........................
..........................
..........................

== L 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
........H1234.............
............5.............
............6.............
............7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 3 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
.........2345.............
........1...6.............
........H...7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== R 17 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
................987654321H
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 10 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s.........98765
.........................4
.........................3
.........................2
.........................1
.........................H

== L 25 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
H123456789................

== U 20 ==

H.........................
1.........................
2.........................
3.........................
4.........................
5.........................
6.........................
7.........................
8.........................
9.........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```

Now, the tail (9) visits 36 positions (including s) at least once:

```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
#.........................
#.............###.........
#............#...#........
.#..........#.....#.......
..#..........#.....#......
...#........#.......#.....
....#......s.........#....
.....#..............#.....
......#............#......
.......#..........#.......
........#........#........
.........########.........
```

Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

In [6]:
# Instantiating values to represent the x- and y-coordinates of the head
x_head, y_head= 0, 0

# Creating a list to hold all the rope positions, starting with 10 empty positions
rope_pos = [[0, 0] for _ in range(10)]


# Creating a list to note all the positions visited
positions_visited = set()
positions_visited.add(tuple(rope_pos[-1]))

# Iterating over all the rope motions
for motion in rope_motions:
    
    # Splitting the direction of the motion from number of units to move
    direction, num_units = motion.split(' ')
    
    # Going through the motions (hee hee)
    for _ in range(int(num_units)):
        
        # Getting the current head position
        x_head, y_head = rope_pos[0][0], rope_pos[0][1]
        
        # Moving the head to its new position
        x_head, y_head = move_head(x_head, y_head, direction)
        
        # Setting the first rope position to be the new head
        rope_pos[0][0], rope_pos[0][1] = x_head, y_head
        
        # Iterating over the tail elements in rope position
        for tail in range(1, 10):
            
            # Checking the change in the current tail vs the previous entry
            dx_tail = rope_pos[tail - 1][0] - rope_pos[tail][0]
            dy_tail = rope_pos[tail - 1][1] - rope_pos[tail][1]
            
            # Checking if the change if either position is greater than 1
            if max(abs(dx_tail), abs(dy_tail)) > 1:
                
                # Incrementing the position appropriately
                rope_pos[tail][0] += dx_tail // abs(dx_tail) if dx_tail else 0
                rope_pos[tail][1] += dy_tail // abs(dy_tail) if dy_tail else 0
                
            # Adding the last position to the set
            positions_visited.add(tuple(rope_pos[-1]))
                
# Printing the final result
print(f'Total number of places visited by the tail: {len(positions_visited)}')

Total number of places visited by the tail: 2327
